In [ ]:
# scp -i aws-sydney.pem carnd@54.206.3.123://home/carnd/data/CarND-Behavioral-Cloning-P3/model.h5 .
# Run 
# (carnd-term1) carnd@ip-172-31-11-20:~$ KERAS_BACKEND=tensorflow 
# if the backend says it use Theano
import csv
from skimage import io
from skimage.color import rgb2gray
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
current_path = 'data/IMG/'
delim = '/'

lines = []
with open('data/driving_log.csv') as csvfile:
#with open('fastest-img/driving_log.csv') as csvfile:    
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

train_samples, validation_samples = train_test_split(lines[1:], test_size=0.2)


def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            measurements = []
            correction = 0.2 # this is a parameter to tune
            for batch_sample in batch_samples:
                image_center_path = batch_sample[0]
                image_left_path = batch_sample[1]
                image_right_path = batch_sample[2]
                
                steering_center = float(batch_sample[3])
                steering_left = steering_center + correction
                steering_right = steering_center - correction
              
                image_center = io.imread(current_path + image_center_path.split(delim)[-1])
                image_left = io.imread(current_path + image_left_path.split(delim)[-1])
                image_right = io.imread(current_path + image_right_path.split(delim)[-1])
                
                images.extend([image_center, image_left, image_right])
                measurements.extend([steering_center, steering_left, steering_right])
                
                images.extend([np.fliplr(image_center), np.fliplr(image_left), np.fliplr(image_right)])
                measurements.extend([-steering_center, -steering_left, -steering_right])
                              
            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(measurements)
                        
            yield shuffle(X_train, y_train)

train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

In [ ]:
def lenet():
    model = Sequential()
    model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)))
    model.add(Lambda(lambda x: x / 255.0 - 0.5))
    model.add(Conv2D(6, (5, 5),activation='relu'))
    model.add(MaxPooling2D())       
    
    model.add(Conv2D(6, (5, 5), activation='relu'))
    model.add(MaxPooling2D())        
    
    model.add(Flatten())
    model.add(Dense(120))    
    model.add(Dense(84))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    return model

def cnn():
    model = Sequential()
    model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
    model.add(Cropping2D(cropping=((50,20), (0,0))))   
    model.add(Conv2D(32, kernel_size=(3, 3),activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1))              
    return model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D,Cropping2D,Lambda
from keras.preprocessing.image import ImageDataGenerator

batch_size = 32

model = lenet()
#model = cnn()

model.compile(loss='mse',optimizer='adam')
model.fit_generator(train_generator, 
                    steps_per_epoch=len(train_samples)//batch_size, 
                    validation_data=validation_generator, 
                    validation_steps=len(validation_samples)//batch_size,
                    epochs=20)
#model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=5)

model.save('model.h5')

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

batch_size = 32
num_classes = 10
epochs = 200
data_augmentation = True

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test))


In [ ]:
from keras.models import load_model

mx = load_model('model.h5')